## <a href='https://projecteuler.net/problem=49'>49. Prime permutations</a>
The arithmetic sequence, 1487, 4817, 8147, in which each of the terms increases by 3330, is unusual in two ways: (i) each of the three terms are prime, and, (ii) each of the 4-digit numbers are permutations of one another.

There are no arithmetic sequences made up of three 1-, 2-, or 3-digit primes, exhibiting this property, but there is one other 4-digit increasing sequence.

What 12-digit number do you form by concatenating the three terms in this sequence?
___

> hints for the question:  
1. find the 3 4-digit primes with the same difference  
2. all these 3 primes contain the same digits of number  
3. only 2 sets, one is given, need to find the other one  
4. answer is the 12-digit number by concatenating thoes 3 primes

> we need prime list:

In [3]:
import math
import numpy as np
import time

In [23]:
def primelist(ub):
    
    # get a list of odd consecutive numbers
    # coz even numbers are not prime, except for 2
    prime = ( np.arange(round(ub/2)) + 1 )*2 - 1
    
    # the first element is 1, replace it as 2
    prime[0] = 2

    checkpos = 0
    while prime[checkpos] < np.sqrt(ub):  # check only upto sqrt of n
        
        # checking the modular (reminder) of each number in the list to a particular number 
        # like [1,2,3] check with 1 , then with 2, then with 3 
        check = prime%(prime[checkpos])   
        
        # keep the checking number
        # a number is divisible by itself (a%a = 0)
        # so in order to keep the 'particular' checking number 
        # make the reminder of it = 1
        # otherwise will be removed in the next step
        # the 'checkpos'th is the prime (checking number) itself 
        # any numbers less than that are not divisible by it
        check[checkpos] = 1
    
        # for those number has the reminder of 0 to the 'particular' checking number, 
        # means they are divisible by the 'particular' checking number
        # so they must not be primes
        # then delete from the list
        prime = np.delete(prime,np.where(check==0)[0])
        # tbh, for large array, avoid using delete, coz this function rather than removing elements, it creates a new one
        ### maybe use list rather than np.array, and pop instead of delete
        
        # go to the next number (next nth ( (n+1)th ) in the list) 
        checkpos += 1

    prime = np.array(prime)
    return prime

> since we need permutation, <br>
copied from q24:

In [87]:
# define Lexicographic permutations
def LP(element,n_th):
    
    di = len(element)
    cnt = 0
    
    while cnt != n_th:
               
        while math.factorial(di) > n_th - cnt and di != 0:
            di -= 1
        
        to = len(element) - di
        head = element[:to]
        tail = element[to:]
        
        if sum(head[-1] < tail) > 0:
            pos = np.where( tail == min( tail[ np.where( tail > head[-1] )[0] ] ) )[0][0]
            head[-1] , tail[pos] = tail[pos] , head[-1]
        
        element = np.append(head,np.sort(tail))
        cnt += math.factorial(di)
        
    return element

In [79]:
n = np.array([1,4,7,8])
for i in range(24):
    print(LP(n,i),end = '')

[1 4 7 8][1 4 8 7][1 7 4 8][1 8 7 4][1 8 4 7][1 8 7 4][4 1 7 8][7 1 8 4][8 4 1 7][8 4 7 1][8 7 1 4][8 7 4 1][8 1 4 7][8 1 7 4][8 4 1 7][8 4 7 1][8 7 1 4][8 7 4 1][8 1 4 7][8 1 7 4][8 4 1 7][8 4 7 1][8 7 1 4][8 7 4 1]

> see, sometimes it doesn't work, coz LP is only for pandigital number <br>
need some modifications, <br>
and LP tells u the n-th permutation, input and output return an array, <br>
but i want all, and returns in str or number

> well i know why it doesn't work <br>
to do for loop with LP <br>
need a re-define of the np.arange after the loop ==> at #@1

In [125]:
# define permutations
def permu(n):
    
# making use of LP
# we know LP only works of pandigital number
# ok no problem
# treat them as the position array

    # 1. make the input into array
    n_str = str(n)
    n_array = np.array([], dtype = 'int32')
    for i in range(len(n_str)):
        n_array = np.append(n_array, int(n_str[i]))
    
    # 2. apply LP, should return position array
    pos_array = np.array([], dtype = 'int32')
    for i in range( math.factorial( len(n_str) ) ):
        
        #@1
        LP_array = np.arange(len(n_str))  
        
        pos_array = np.append(pos_array, LP(LP_array,i)).reshape( (i+1) , len(n_str) )
        
    # 3. permutation according to the position array
    permu_array = np.array([], dtype = 'int32')
    for i in range( len(pos_array) ):
        
        permu_i = ''
        for j in range( len(n_str) ):
            permu_i += n_str[ pos_array[i][j] ]

        permu_array = np.append( permu_array , int(permu_i) )
        
    return permu_array

In [127]:
permu(1487)

array([1487, 1478, 1847, 1874, 1748, 1784, 4187, 4178, 4817, 4871, 4718,
       4781, 8147, 8174, 8417, 8471, 8714, 8741, 7148, 7184, 7418, 7481,
       7814, 7841])

> now we have all we need:


In [150]:
# get primes from 1000 to 9999
limit = 10000
p_to10000 = primelist(limit)

p1000 = p_to10000[(np.where(p_to10000 > 1000)[0][0]):]

# set operation
p1000_set = set(p1000)

# use while loop can stop immediately after we find the 2nd set
# counting
cnt = 0

# define the position in primelist
pos = 0

t1 = time.time()    
while cnt < 2:
    
    # for each prime, go thru permu
    check_prime = permu(p1000[pos])
    
    # remove the non-prime
    permu_prime = np.array([], dtype = 'int32')
    for i in range( len( check_prime ) ):
        if check_prime[i] in p1000_set:
            permu_prime = np.append( permu_prime , check_prime[i] )
            
    # now we can check the difference
    # pick any 3 from those
    
    pick = 3
    
    pi = len(permu_prime) - pick + 1  # coz must not choose the last 2
    pj = len(permu_prime) - pick + 2  # coz must not choose the last 1
    pk = len(permu_prime) - pick + 3  # coz must not choose the last 0, well it is the whole len
    
    for i in range(pi): 
        d1 = permu_prime[i]
        
        for j in range(i,pj):
            d2 = permu_prime[j]
        
            for k in range(j,pk):
                d3 = permu_prime[k]
                
                # check difference
                if d2 - d1 == d3 - d2 and  d1 < d2 < d3 : 
                    print(d1,d2,d3)
                    cnt += 1
                
                # stop switch    
                if cnt == 2:
                    break
                        
    pos += 1
    
    # stop switch
    if pos == len(p1000):
        break
                
t2 = time.time()
print(t2-t1,'seconds taken')

1487 4817 8147
2969 6299 9629
0.3401186466217041 seconds taken


> yup, 296962999629 is correct